## Level 6: PyQt5 GUI User Management Application

This code provides a complete replacement for the previous console-based script, offering a graphical user interface (GUI) for user data management. It leverages PyQt5 to create a windowed application.

### Key Features:

* **Loads Data from CSV:**
    * Reads user data from `user_data.csv` into a Pandas DataFrame.
* **Displays Data in a Table:**
    * Shows the DataFrame's contents in a `QTableWidget` for easy viewing and interaction.
* **Provides Add, Delete, and Replace Functionality:**
    * **Add User:**
        * Opens a dialog to input new user information.
        * Generates a unique alphanumeric ID for the new user.
        * Adds the user's data to the DataFrame.
        * Saves the updated DataFrame to the CSV file.
    * **Delete User:**
        * Deletes the selected rows from the DataFrame.
        * Saves the changes to the CSV file.
    * **Replace User:**
        * Opens a dialog to edit the selected user's information.
        * Updates the DataFrame with the edited information.
        * Saves the changes to the CSV file.
* **Updates Table Automatically:**
    * Refreshes the `QTableWidget` after each data modification to reflect the latest changes.

### How to Run:

1.  **Install PyQt5:**
    * If you don't have it installed, use `pip install PyQt5` in your terminal or command prompt.
2.  **Run the Script:**
    * Execute the Python script from your terminal or your preferred IDE.

### Key Components:

* **`UserForm` Class:**
    * Creates a dialog window for adding or editing user information.
    * Utilizes `QFormLayout` for an organized and user-friendly layout.
* **`UserManagementApp` Class:**
    * Creates the main application window.
    * Uses a `QTableWidget` to display the user data.
    * Implements the core functionality for adding, deleting, and replacing user entries.
    * Handles the loading and saving of data to the `user_data.csv` file.
* **`load_user_data()` and `save_user_data()` Functions:**
    * These functions handle the reading and writing of data to the CSV file, leveraging the power of Pandas for efficient data manipulation.
* **`generate_user_id()` Function:**
    * Generates unique alphanumeric IDs for each user, ensuring data integrity.

This application provides a user-friendly and efficient way to manage user data using a graphical interface.

In [2]:
import pandas as pd
import os
import random
import string
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QLineEdit, QPushButton,
                             QVBoxLayout, QHBoxLayout, QTableWidget, QTableWidgetItem,
                             QMessageBox, QDialog, QFormLayout)

CSV_FILE = "user_data.csv"

def generate_user_id(existing_ids):
    """Generates a random alphanumeric user ID, ensuring uniqueness."""
    while True:
        characters = string.ascii_letters + string.digits
        user_id = ''.join(random.choice(characters) for i in range(7))
        if user_id not in existing_ids:
            return user_id

def load_user_data():
    """Loads user data from the CSV file."""
    if os.path.exists(CSV_FILE):
        try:
            return pd.read_csv(CSV_FILE)
        except pd.errors.EmptyDataError:
            return pd.DataFrame()
    else:
        return pd.DataFrame()

def save_user_data(df):
    """Saves user data to the CSV file."""
    df.to_csv(CSV_FILE, index=False)

class UserForm(QDialog):
    def __init__(self, parent=None, user_data=None):
        super().__init__(parent)
        self.user_data = user_data
        self.initUI()

    def initUI(self):
        layout = QFormLayout()

        self.first_name_edit = QLineEdit()
        self.last_name_edit = QLineEdit()
        self.age_edit = QLineEdit()
        self.city_edit = QLineEdit()
        self.occupation_edit = QLineEdit()
        self.favorite_food_edit = QLineEdit()

        layout.addRow("First Name:", self.first_name_edit)
        layout.addRow("Last Name:", self.last_name_edit)
        layout.addRow("Age:", self.age_edit)
        layout.addRow("City:", self.city_edit)
        layout.addRow("Occupation:", self.occupation_edit)
        layout.addRow("Favorite Food:", self.favorite_food_edit)

        buttons_layout = QHBoxLayout()
        self.save_button = QPushButton("Save")
        self.save_button.clicked.connect(self.accept)
        buttons_layout.addWidget(self.save_button)

        layout.addRow(buttons_layout)
        self.setLayout(layout)

    def get_user_info(self):
        return {
            "first_name": self.first_name_edit.text(),
            "last_name": self.last_name_edit.text(),
            "age": self.age_edit.text(),
            "city": self.city_edit.text(),
            "occupation": self.occupation_edit.text(),
            "favorite_food": self.favorite_food_edit.text()
        }

class UserManagementApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.load_data()

    def initUI(self):
        layout = QVBoxLayout()

        self.table = QTableWidget()
        layout.addWidget(self.table)

        buttons_layout = QHBoxLayout()
        self.add_button = QPushButton("Add User")
        self.add_button.clicked.connect(self.add_user)
        buttons_layout.addWidget(self.add_button)

        self.delete_button = QPushButton("Delete User")
        self.delete_button.clicked.connect(self.delete_user)
        buttons_layout.addWidget(self.delete_button)

        self.replace_button = QPushButton("Replace User")
        self.replace_button.clicked.connect(self.replace_user)
        buttons_layout.addWidget(self.replace_button)

        layout.addLayout(buttons_layout)
        self.setLayout(layout)
        self.setWindowTitle("User Management")

    def load_data(self):
        self.df = load_user_data()
        self.update_table()

    def update_table(self):
        self.table.setRowCount(len(self.df))
        self.table.setColumnCount(len(self.df.columns))
        self.table.setHorizontalHeaderLabels(self.df.columns)

        for row_index, row in self.df.iterrows():
            for col_index, value in enumerate(row):
                self.table.setItem(row_index, col_index, QTableWidgetItem(str(value)))

    def add_user(self):
        user_form = UserForm(self)
        if user_form.exec_():
            user_info = user_form.get_user_info()
            existing_ids = self.df['id'].tolist() if not self.df.empty else []
            user_id = generate_user_id(existing_ids)
            user_info['id'] = user_id
            self.df = pd.concat([self.df, pd.DataFrame([user_info])], ignore_index=True)
            save_user_data(self.df)
            self.load_data()

    def delete_user(self):
        selected_rows = self.table.selectionModel().selectedRows()
        if not selected_rows:
            return
        row_indices = sorted([index.row() for index in selected_rows], reverse=True)
        for row_index in row_indices:
            self.df = self.df.drop(row_index).reset_index(drop=True)
        save_user_data(self.df)
        self.load_data()

    def replace_user(self):
        selected_rows = self.table.selectionModel().selectedRows()
        if not selected_rows or len(selected_rows) > 1:
            return
        row_index = selected_rows[0].row()
        user_data = self.df.iloc[row_index].to_dict()
        user_form = UserForm(self, user_data)
        user_form.first_name_edit.setText(user_data['first_name'])
        user_form.last_name_edit.setText(user_data['last_name'])
        user_form.age_edit.setText(str(user_data['age']))
        user_form.city_edit.setText(user_data['city'])
        user_form.occupation_edit.setText(user_data['occupation'])
        user_form.favorite_food_edit.setText(user_data['favorite_food'])
        if user_form.exec_():
            new_user_info = user_form.get_user_info()
            for key, value in new_user_info.items():
                self.df.loc[row_index, key] = value
            save_user_data(self.df)
            self.load_data()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = UserManagementApp()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Dropping the Index Column from a CSV File in Jupyter Notebook

This code provides a function to remove the "Unnamed: 0" index column from a CSV file directly within a Jupyter Notebook environment.

### Explanation:

**1. Importing Libraries:**

* **`pandas`:** This library is essential for data manipulation, specifically for reading and writing CSV files.
* **`os`:** This library is used to interact with the operating system, allowing us to check if a file exists before attempting to process it.

**2. `drop_index_column(csv_file)` Function:**

* **Input:** The function takes the path to the CSV file (`csv_file`) as an argument.
* **File Existence Check:**
    * `if os.path.exists(csv_file):` This line verifies that the specified file exists before attempting to read it, preventing errors if the file is missing.
* **Error Handling:**
    * A `try...except` block is implemented to gracefully handle potential errors during file processing:
        * **`FileNotFoundError`:** Catches errors if the specified file does not exist.
        * **`pd.errors.EmptyDataError`:** Catches errors if the CSV file is empty.
        * **`Exception`:** Catches any other unexpected errors that might occur.
* **Reading the CSV:**
    * `df = pd.read_csv(csv_file)`: If the file exists, this line reads the CSV data into a Pandas DataFrame, making it easy to manipulate.
* **Checking for the Index Column:**
    * `if "Unnamed: 0" in df.columns:`: This line checks if the DataFrame contains a column named "Unnamed: 0," which is typically the index column created when a DataFrame is saved to CSV with an index.
* **Dropping the Index Column:**
    * `df = df.drop("Unnamed: 0", axis=1)`: If the "Unnamed: 0" column exists, this line removes it from the DataFrame.
* **Saving the Modified CSV:**
    * `df.to_csv(csv_file, index=False)`: This line saves the modified DataFrame back to the original CSV file, ensuring that the index is not written again. `index=False` is crucial to prevent the index from reappearing.
* **Printing Messages:**
    * The function includes `print()` statements to provide feedback to the user, indicating whether the column was dropped, if it was not found, or if an error occurred.

**3. Example Usage:**

* `drop_index_column(CSV_FILE)`: This line calls the function, passing the `user_data.csv` file path (defined by `CSV_FILE`) as an argument. This will execute the function and attempt to remove the index column from the specified file.

**How to Use in Jupyter Notebook:**

1.  Copy and paste the code into a Jupyter Notebook cell.
2.  Ensure that `user_data.csv` is in the same directory as your notebook or provide the full file path.
3.  Run the cell by pressing `Shift + Enter`.
4.  Check the output messages to confirm the operation's success.
5.  Verify the changes by examining the `user_data.csv` file.

In [ ]:
import pandas as pd
import os

CSV_FILE = "user_data.csv"

def drop_index_column(csv_file):
    """Drops the index column from a CSV file if it exists."""
    if os.path.exists(csv_file):
        try:
            df = pd.read_csv(csv_file)
            if "Unnamed: 0" in df.columns:
                df = df.drop("Unnamed: 0", axis=1)
                df.to_csv(csv_file, index=False)
                print(f"Index column 'Unnamed: 0' dropped from {csv_file}.")
            else:
                print(f"Index column 'Unnamed: 0' not found in {csv_file}.")
        except FileNotFoundError:
            print(f"Error: File '{csv_file}' not found.")
        except pd.errors.EmptyDataError:
            print(f"Error: '{csv_file}' is empty.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
    else:
        print(f"Error: File '{csv_file}' not found.")

# Example usage:
drop_index_column(CSV_FILE)

## Enhancements to the PyQt5 User Management App

###  CSV Saving:

* It's important to confirm that the script *does* correctly save changes back to the CSV file.
* Every time you add, delete, or replace a user, the `save_user_data(self.df)` function is called, ensuring that the `user_data.csv` file is updated with the latest information.

### Adding an "Exit" Button:

* An "Exit" button has been added to the application's interface.
* This button, when clicked, will close the application window.
* In the `initUI()` function, the button is created with `QPushButton("Exit")`, and its click event is connected to `self.close()` using `self.exit_button.clicked.connect(self.close)`.

### Summary of Changes:

* The `load_data()` function now includes `self.df.reset_index(drop=True)` to remove the index column from the displayed table.
* Confirmation that the script correctly writes to the CSV file is included.
* An exit button has been added.

The code has been updated to reflect these enhancements, providing a more user-friendly and complete application.

In [ ]:
import pandas as pd
import os
import random
import string
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QLineEdit, QPushButton,
                             QVBoxLayout, QHBoxLayout, QTableWidget, QTableWidgetItem,
                             QMessageBox, QDialog, QFormLayout)

CSV_FILE = "user_data.csv"

def generate_user_id(existing_ids):
    """Generates a random alphanumeric user ID, ensuring uniqueness."""
    while True:
        characters = string.ascii_letters + string.digits
        user_id = ''.join(random.choice(characters) for i in range(7))
        if user_id not in existing_ids:
            return user_id

def load_user_data():
    """Loads user data from the CSV file."""
    if os.path.exists(CSV_FILE):
        try:
            return pd.read_csv(CSV_FILE)
        except pd.errors.EmptyDataError:
            return pd.DataFrame()
    else:
        return pd.DataFrame()

def save_user_data(df):
    """Saves user data to the CSV file."""
    df.to_csv(CSV_FILE, index=False)

class UserForm(QDialog):
    def __init__(self, parent=None, user_data=None):
        super().__init__(parent)
        self.user_data = user_data
        self.initUI()

    def initUI(self):
        layout = QFormLayout()

        self.first_name_edit = QLineEdit()
        self.last_name_edit = QLineEdit()
        self.age_edit = QLineEdit()
        self.city_edit = QLineEdit()
        self.occupation_edit = QLineEdit()
        self.favorite_food_edit = QLineEdit()

        layout.addRow("First Name:", self.first_name_edit)
        layout.addRow("Last Name:", self.last_name_edit)
        layout.addRow("Age:", self.age_edit)
        layout.addRow("City:", self.city_edit)
        layout.addRow("Occupation:", self.occupation_edit)
        layout.addRow("Favorite Food:", self.favorite_food_edit)

        buttons_layout = QHBoxLayout()
        self.save_button = QPushButton("Save")
        self.save_button.clicked.connect(self.accept)
        buttons_layout.addWidget(self.save_button)

        layout.addRow(buttons_layout)
        self.setLayout(layout)

    def get_user_info(self):
        return {
            "first_name": self.first_name_edit.text(),
            "last_name": self.last_name_edit.text(),
            "age": self.age_edit.text(),
            "city": self.city_edit.text(),
            "occupation": self.occupation_edit.text(),
            "favorite_food": self.favorite_food_edit.text()
        }

class UserManagementApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.load_data()

    def initUI(self):
        layout = QVBoxLayout()

        self.table = QTableWidget()
        layout.addWidget(self.table)

        buttons_layout = QHBoxLayout()
        self.add_button = QPushButton("Add User")
        self.add_button.clicked.connect(self.add_user)
        buttons_layout.addWidget(self.add_button)

        self.delete_button = QPushButton("Delete User")
        self.delete_button.clicked.connect(self.delete_user)
        buttons_layout.addWidget(self.delete_button)

        self.replace_button = QPushButton("Replace User")
        self.replace_button.clicked.connect(self.replace_user)
        buttons_layout.addWidget(self.replace_button)
        
        self.exit_button = QPushButton("Exit")
        self.exit_button.clicked.connect(self.close)
        buttons_layout.addWidget(self.exit_button)

        layout.addLayout(buttons_layout)
        self.setLayout(layout)
        self.setWindowTitle("User Management")

    def load_data(self):
        self.df = load_user_data()
        self.df = self.df.reset_index(drop=True) #reset the index before loading to the table
        self.update_table()

    def update_table(self):
        self.table.setRowCount(len(self.df))
        self.table.setColumnCount(len(self.df.columns))
        self.table.setHorizontalHeaderLabels(self.df.columns)

        for row_index, row in self.df.iterrows():
            for col_index, value in enumerate(row):
                self.table.setItem(row_index, col_index, QTableWidgetItem(str(value)))

    def add_user(self):
        user_form = UserForm(self)
        if user_form.exec_():
            user_info = user_form.get_user_info()
            existing_ids = self.df['id'].tolist() if not self.df.empty else []
            user_id = generate_user_id(existing_ids)
            user_info['id'] = user_id
            self.df = pd.concat([self.df, pd.DataFrame([user_info])], ignore_index=True)
            save_user_data(self.df)
            self.load_data()

    def delete_user(self):
        selected_rows = self.table.selectionModel().selectedRows()
        if not selected_rows:
            return
        row_indices = sorted([index.row() for index in selected_rows], reverse=True)
        for row_index in row_indices:
            self.df = self.df.drop(row_index).reset_index(drop=True)
        save_user_data(self.df)
        self.load_data()

    def replace_user(self):
        selected_rows = self.table.selectionModel().selectedRows()
        if not selected_rows or len(selected_rows) > 1:
            return
        row_index = selected_rows[0].row()
        user_data = self.df.iloc[row_index].to_dict()
        user_form = UserForm(self, user_data)
        user_form.first_name_edit.setText(user_data['first_name'])
        user_form.last_name_edit.setText(user_data['last_name'])
        user_form.age_edit.setText(str(user_data['age']))
        user_form.city_edit.setText(user_data['city'])
        user_form.occupation_edit.setText(user_data['occupation'])
        user_form.favorite_food_edit.setText(user_data['favorite_food'])
        if user_form.exec_():
            new_user_info = user_form.get_user_info()
            for key, value in new_user_info.items():
                self.df.loc[row_index, key] = value
            save_user_data(self.df)
            self.load_data()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = UserManagementApp()
    window.show()
    sys.exit(app.exec_())